In [ ]:
from flask import Flask, request, jsonify
from sentence_transformers import SentenceTransformer, util
import mysql.connector
import os
from PyPDF2 import PdfReader



app = Flask(__name__)

# Modelin yüklenmesi
model_name = "Sakil/sentence_similarity_semantic_search"
model = SentenceTransformer(model_name)


# PDF dosyalarının bulunduğu dizin
pdf_directory = r'C:\Users\yasemin\Desktop\pdfSoftware'

# Tüm PDF dosyalarını al
pdf_files = [file for file in os.listdir(pdf_directory) if file.endswith('.pdf')]

# Tüm PDF dosyalarının içeriklerini saklamak için bir liste oluştur
pdf_contents = []

# Her PDF dosyasının içeriğini okuyup listeye ekle
for pdf_file in pdf_files:
    file_path = os.path.join(pdf_directory, pdf_file)
    with open(file_path, 'rb') as file:
        reader = PdfReader(file)
        content = ""
        for page_num in range(len(reader.pages)):
            content += reader.pages[page_num].extract_text()
        pdf_contents.append(content)




# Her PDF dosyasının içeriğini içeren listeyi yazdır
for i, content in enumerate(pdf_contents):
    print(f"PDF {i+1} içeriği:\n{content}\n")

# Örnek cümleler
sentences =  pdf_contents



# MySQL bağlantı bilgileri
config = {
  'user': 'root',
  'password': 'Yasoyaso1818',
  'host': '127.0.0.1',
  'database': 'softwaresolution_app',
  'raise_on_warnings': True
}

# MySQL bağlantısını oluştur
cnx = mysql.connector.connect(**config)
cnx2 = mysql.connector.connect(**config)





# Cursor oluştur
cursor = cnx.cursor()
cursor2 = cnx2.cursor()



# Sorguyu tanımla
query = """
     SELECT requirements, COUNT(*) AS count
FROM softwareselectionhistory
GROUP BY requirements
HAVING COUNT(*) > 1;
"""

query2 = """
    SELECT description
    FROM softwaresolution
    """

@app.route('/semantic-search', methods=['GET'])
def semantic_search():
    # Sorguyu çalıştır
    cursor.execute(query)

    # Sonucu al
    result = cursor.fetchone()

    # Sorguyu çalıştır
    cursor2.execute(query2)
    result2 = cursor2.fetchone()

    # Tüm descriptionları içeren bir liste oluştur
    descriptions = [row[0] for row in cursor2.fetchall()]
    print(result2)
   

   # Sonucu yazdır
    if result:
        print("En çok tekrar eden değere ait çözüm:", result[0])
    else:
        print("Kayıt bulunamadı.")

 
    # Bağlantıyı kapat
    cursor.close()
    cnx.close()

      # Bağlantıyı kapat
    cursor2.close()
    cnx2.close()
   
   
    # Giriş cümlesinin kodlanması
    input_embedding = model.encode( result , convert_to_tensor=True)
   
    # Diğer tüm cümlelerin kodlanması
    sentence_embeddings = model.encode(sentences, convert_to_tensor=True)
   
    # Cosine benzerliğinin hesaplanması
    cos_scores = util.pytorch_cos_sim(input_embedding, sentence_embeddings)[0]
   
    # Benzerlik skorlarına göre sıralama
    sorted_indices = cos_scores.argsort(descending=True)

    # En büyük benzerlik skoruna sahip olanın bulunması
    max_score_index = cos_scores.argmax()
    max_score_sentence = sentences[max_score_index]
    print(max_score_sentence)
   
    # Benzerlik sırasına göre cümlelerin oluşturulması
    sorted_sentences = [sentences[idx] for idx in sorted_indices]
    sorted_scores = [float(cos_scores[idx]) for idx in sorted_indices]

    #butun onceki kodlarin hepsi gelecek input:sorted_sentences olacak
      
    # JSON yanıtının oluşturulması
    response = {
        "max_score_sentence":  max_score_sentence
    }

    # Tüm descriptionları sentences listesi gibi bir string dizisine dönüştür
    sentences2 = descriptions

    # İstek verisinin alınması

    input_sentence2 =max_score_sentence
    input_embedding2 = model.encode(input_sentence2, convert_to_tensor=True)
   
    # Diğer tüm cümlelerin kodlanması
    sentence_embeddings2 = model.encode(sentences2, convert_to_tensor=True)
   
     
    # Cosine benzerliğinin hesaplanması
    cos_scores2 = util.pytorch_cos_sim(input_embedding2, sentence_embeddings2)[0]
   
    # Benzerlik skorlarına göre sıralama
    sorted_indices2 = cos_scores2.argsort(descending=True)

    # Benzerlik sırasına göre cümlelerin oluşturulması
    sorted_sentences2 = [sentences2[idx] for idx in sorted_indices2]
    sorted_scores2 = [float(cos_scores2[idx]) for idx in sorted_indices2]
   
    # JSON yanıtının oluşturulması
    response2 = {
        "input_sentence": input_sentence2,
        "sorted_sentences": sorted_sentences2,
        "sorted_scores": sorted_scores2
    }

    cnx3 = mysql.connector.connect(**config)
    # Cursor oluştur
    cursor3 = cnx3.cursor()

    # Tüm descriptionları al
    # Python kodunda sorted_sentences listesini bir dize olarak oluşturma
    sorted_sentences_str = ", ".join([f"'{sentence}'" for sentence in sorted_sentences])
    print(sorted_sentences_str)

    # Listeyi stringe dönüştürme
    sorted_sentences_str = str(sorted_sentences2)

    # Köşeli parantezleri kaldırma
    sorted_sentences_cleaned = sorted_sentences_str.strip("[]")


    query3 = f"""
    SELECT ID,   requirement, provider_answer, provider_explanation
    FROM softwaresolution
    WHERE description IN ({    sorted_sentences_cleaned})
    """

    print(query3)

    cursor3.execute(query3)

    result3=cursor3.fetchall()

    # Bağlantıyı kapat
    cursor3.close()
    cnx3.close()

    # Düzenlenmiş veri listesi
    formatted_data = []

    # Her bir veri öğesini düzenle
    for item in result3:
        formatted_item = {
        "softwareSolutionId": item[0],
        "requirement": item[1],
        "providerAnswer": item[2],
        "providerExplanation": item[3]
         }
        formatted_data.append(formatted_item)

   
      
    return jsonify(formatted_data)

if __name__ == '__main__':
    app.run(host='0.0.0.0',port=8005)

PDF 1 içeriği:
Software Functions: Data Analysis Tools 
This PDF document explains the general functions and uses of data analysis software. The document 
discusses basic functions such as data visualization, report generation, trend analysis, etc., as well as 
special features and advantages provided by different software. It also offers tips and techniques to 
optimize data analysis processes. 

PDF 2 içeriği:
Familiar Software Forms: Business Accounting 
This PDF document explains common software forms used for business accounting. In addition to basic 
forms that facilitate operations such as creating invoices, preparing an income-expense statement, and  
filling out tax returns, it offers suggestions and practical information to help businesses manage  their 
financial processes. 

PDF 3 içeriği:
Training Guide: Project Management Software 
This PDF document provides a training guide for using project management software effectively. The 
document covers the key features of the so

 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8005
 * Running on http://192.168.2.114:8005
Press CTRL+C to quit


('Use the central overview to keep an eye on all selection initiatives in the organization and their status.',)
En çok tekrar eden değere ait çözüm: [Scalable cloud hosting, Website performance optimization, Load testing]
Familiar Software Forms: Business Accounting 
This PDF document explains common software forms used for business accounting. In addition to basic 
forms that facilitate operations such as creating invoices, preparing an income-expense statement, and  
filling out tax returns, it offers suggestions and practical information to help businesses manage  their 
financial processes. 


192.168.2.114 - - [13/Apr/2024 12:39:24] "GET /semantic-search HTTP/1.1" 200 -


'Familiar Software Forms: Business Accounting 
This PDF document explains common software forms used for business accounting. In addition to basic 
forms that facilitate operations such as creating invoices, preparing an income-expense statement, and  
filling out tax returns, it offers suggestions and practical information to help businesses manage  their 
financial processes. ', 'Training Guide: Project Management Software 
This PDF document provides a training guide for using project management software effectively. The 
document covers the key features of the software for planning, monitoring and evaluating projects. It 
also includes project management best practices and step- by-step instructions on how to use the tools 
the software offers. ', 'Product Guide: Robotic Vacuum Cleaner 
This PDF document explains step by step how users can set up and use a new robotic vacuum cleaner. 
The document covers the main features of the device in detail, such as assembly, charging, cleaning